## Premier League Data Pipeline

In [41]:
# Definicion de Librerias

import requests
import pandas as pd
from configparser import ConfigParser
from utils import get_data,get_metadata_from_json,update_last_update_in_json
from deltalake import DeltaTable,write_deltalake
from datetime import datetime,timedelta
import os
import pyarrow as pa
import json
import great_expectations as gx




In [9]:
# Definicion de Variables de configuracion y de URL 
parser = ConfigParser()
parser.read('pipeline.conf')
api_key = parser.items('football-data')[0][1]
headers = {'X-AUTH-TOKEN':api_key}

url_base = 'https://api.football-data.org/v4/'

In [10]:
# Chequeo de la conexion a la API

status_code = requests.get('https://api.football-data.org/v4/matches', headers=headers).status_code
if status_code == 200:
    print('Prueba de Conexion Exitosa')
else:
    print(f'Error en la conexion, codigo: {status_code}')
    

Prueba de Conexion Exitosa


### Extraccion full de los equipos de la **Premier League** de la temporada 2024/2025

In [11]:
endpoint_teams = 'competitions/PL/teams'

data_PL = get_data(url_base,endpoint_teams,headers=headers)
data_PL.keys()

dict_keys(['count', 'filters', 'competition', 'season', 'teams'])

In [12]:
teams_data = data_PL['teams']
teams_data[0].keys()

dict_keys(['area', 'id', 'name', 'shortName', 'tla', 'crest', 'address', 'website', 'founded', 'clubColors', 'venue', 'runningCompetitions', 'coach', 'squad', 'staff', 'lastUpdated'])

In [53]:
# Creacion de un DataFrame con la informacion que queremos de los equipos
# mediante list comprenhesion
teams = [
    dict(
            id=team['id'],
            team=team['name'],
            Stadium=team['venue'],
            dt=f"{team['coach']['firstName']} {team['coach']['lastName']}",
            lastUpdated=team['lastUpdated']
        )
for team in teams_data]

teams[0:2]

[{'id': 57,
  'team': 'Arsenal FC',
  'Stadium': 'Emirates Stadium',
  'dt': ' Mikel Arteta',
  'lastUpdated': '2022-02-10T19:48:56Z'},
 {'id': 58,
  'team': 'Aston Villa FC',
  'Stadium': 'Villa Park',
  'dt': ' Unai Emery',
  'lastUpdated': '2022-04-03T16:22:14Z'}]

In [14]:
columns = ['id','team','Stadium','dt','lastUpdated']
df_2024_2025_PL = pd.DataFrame(data=teams,columns=columns)
# id , team , Stadium , dt , LastUpdated
df_2024_2025_PL.head() # -> DataFrame con la informacion de los equipos de la Premier League

,id,team,Stadium,dt,lastUpdated
0,57,Arsenal FC,Emirates Stadium,Mikel Arteta,2022-02-10T19:48:56Z
1,58,Aston Villa FC,Villa Park,Unai Emery,2022-04-03T16:22:14Z
2,61,Chelsea FC,Stamford Bridge,Enzo Maresca,2022-02-10T19:24:40Z
3,62,Everton FC,Goodison Park,Sean Dyche,2022-02-10T19:47:42Z
4,63,Fulham FC,Craven Cottage,Marco Silva,2024-07-29T17:16:11Z


In [15]:
# Se sobreescriben todos los datos
write_deltalake(
    "data_lake/teams",
    df_2024_2025_PL,
    mode="overwrite"
)

In [16]:
teams_dt = DeltaTable("data_lake/teams").to_pandas()
teams_dt.head()

,id,team,Stadium,dt,lastUpdated
0,57,Arsenal FC,Emirates Stadium,Mikel Arteta,2022-02-10T19:48:56Z
1,58,Aston Villa FC,Villa Park,Unai Emery,2022-04-03T16:22:14Z
2,61,Chelsea FC,Stamford Bridge,Enzo Maresca,2022-02-10T19:24:40Z
3,62,Everton FC,Goodison Park,Sean Dyche,2022-02-10T19:47:42Z
4,63,Fulham FC,Craven Cottage,Marco Silva,2024-07-29T17:16:11Z


### Extraccion full de las plantillas de los equipos de la premier league de la temporada 2024/2025

Todavia no tienen un uso definido, pero sirven como datos estaticos (o de poca ocurrencia de actualizacion), ya que las plantillas solo cambian en el mercado invernal y en el mercado de verano.

In [17]:
teams_data[0]['squad'][0].keys()

dict_keys(['id', 'name', 'position', 'dateOfBirth', 'nationality'])

In [74]:
# Guardamos la informacion que necesitemos de todos los equipos
# List Comprenhension anidado
    
squads = [
    dict(id_team=team['id'],
         squad=[
            dict(
                id=player_data['id'],
                name=player_data['name'],
                position=player_data['position'],
                nationality=player_data['nationality'],
                dateOfBirth=player_data['dateOfBirth']
                )
            for player_data in team['squad']
                ]
        )
    for team in teams_data
]

# Este dataframe contiene por cada equipo -> (id del equipo, lista de jugadores)    
squads[0]['squad'][5:7]

[{'id': 9034,
  'name': 'Takehiro Tomiyasu',
  'position': 'Defence',
  'nationality': 'Japan',
  'dateOfBirth': '1998-11-05'},
 {'id': 15615,
  'name': 'Kieran Tierney',
  'position': 'Defence',
  'nationality': 'Scotland',
  'dateOfBirth': '1997-06-05'}]

In [75]:
df_squads = pd.DataFrame(data=squads)
df_squads.head(10)
# Cada registro muestra el id_team y una lista de jugadores


,id_team,squad
0,57,"[{'id': 4832, 'name': 'David Raya', 'position'..."
1,58,"[{'id': 3141, 'name': 'Emiliano Martínez', 'po..."
2,61,"[{'id': 3189, 'name': 'Kepa Arrizabalaga', 'po..."
3,62,"[{'id': 3309, 'name': 'Jordan Pickford', 'posi..."
4,63,"[{'id': 3174, 'name': 'Bernd Leno', 'position'..."
5,64,"[{'id': 1795, 'name': 'Alisson', 'position': '..."
6,65,"[{'id': 3222, 'name': 'Ederson', 'position': '..."
7,66,"[{'id': 7544, 'name': 'André Onana', 'position..."
8,67,"[{'id': 3310, 'name': 'Nick Pope', 'position':..."
9,73,"[{'id': 3086, 'name': 'Guglielmo Vicario', 'po..."


In [76]:
# Normalizamos el json para que cada jugador sea un registro con su respectivo id_team, y sea mas
df_squads = pd.json_normalize(data=squads,record_path='squad',meta='id_team')
df_squads.head()

,id,name,position,nationality,dateOfBirth,id_team
0,4832,David Raya,Goalkeeper,Spain,1995-09-15,57
1,5530,Aaron Ramsdale,Goalkeeper,England,1998-05-14,57
2,153843,Karl Jakob Hein,Goalkeeper,Estonia,2002-04-13,57
3,6154,Ben White,Defence,England,1997-10-08,57
4,7889,Oleksandr Zinchenko,Defence,Ukraine,1996-12-15,57


In [77]:
# Guardar en deltalake particionado por id_team
# Cada parquet contiene la plantilla del equipo
write_deltalake(
    "data_lake/squads",
    df_squads,
    mode="overwrite",
    partition_by="id_team"
)

### Extraccion full de los partidos de la premier league de la temporada 2024/2025
Para tener un dataset completo de la temporada 2024/2025 de la premier league, se extraen todos los partidos de la temporada.

In [78]:
endpoint_matches = 'competitions/PL/matches'
data_matches = get_data(url_base,endpoint_matches,headers=headers)
data_matches['matches'][0]


{'area': {'id': 2072,
  'name': 'England',
  'code': 'ENG',
  'flag': 'https://crests.football-data.org/770.svg'},
 'competition': {'id': 2021,
  'name': 'Premier League',
  'code': 'PL',
  'type': 'LEAGUE',
  'emblem': 'https://crests.football-data.org/PL.png'},
 'season': {'id': 2287,
  'startDate': '2024-08-16',
  'endDate': '2025-05-25',
  'currentMatchday': 1,
  'winner': None},
 'id': 497410,
 'utcDate': '2024-08-16T19:00:00Z',
 'status': 'TIMED',
 'matchday': 1,
 'stage': 'REGULAR_SEASON',
 'group': None,
 'lastUpdated': '2024-07-23T10:21:24Z',
 'homeTeam': {'id': 66,
  'name': 'Manchester United FC',
  'shortName': 'Man United',
  'tla': 'MUN',
  'crest': 'https://crests.football-data.org/66.png'},
 'awayTeam': {'id': 63,
  'name': 'Fulham FC',
  'shortName': 'Fulham',
  'tla': 'FUL',
  'crest': 'https://crests.football-data.org/63.png'},
 'score': {'winner': None,
  'duration': 'REGULAR',
  'fullTime': {'home': None, 'away': None},
  'halfTime': {'home': None, 'away': None}},


In [79]:
# Estado de todos los partidos matchday, id, status, utcDate


matches_data = [
    dict(
            matchday=match['matchday'],
            id=match['id'],
            status=match['status'],
            date=match['utcDate'],
            home=match['homeTeam']['id'],
            away=match['awayTeam']['id'],
            goals_home=match['score']['fullTime']['home'],
            goals_away=match['score']['fullTime']['away']
    )
    for match in data_matches['matches']
]    


matches_data = pd.DataFrame(data=matches_data)
matches_data.head()


,matchday,id,status,date,home,away,goals_home,goals_away
0,1,497410,TIMED,2024-08-16T19:00:00Z,66,63,None,None
1,1,497411,TIMED,2024-08-17T11:30:00Z,349,64,None,None
2,1,497412,TIMED,2024-08-17T14:00:00Z,57,76,None,None
3,1,497413,TIMED,2024-08-17T14:00:00Z,62,397,None,None
4,1,497414,TIMED,2024-08-17T14:00:00Z,67,340,None,None


In [24]:
matches_data.fillna(0,inplace=True)
matches_data

,matchday,id,status,date,home,away,goals_home,goals_away
0,1,497410,TIMED,2024-08-16T19:00:00Z,66,63,0,0
1,1,497411,TIMED,2024-08-17T11:30:00Z,349,64,0,0
2,1,497412,TIMED,2024-08-17T14:00:00Z,57,76,0,0
3,1,497413,TIMED,2024-08-17T14:00:00Z,62,397,0,0
4,1,497414,TIMED,2024-08-17T14:00:00Z,67,340,0,0
...,...,...,...,...,...,...,...,...
375,38,497785,TIMED,2025-05-25T15:00:00Z,67,62,0,0
376,38,497786,TIMED,2025-05-25T15:00:00Z,351,61,0,0
377,38,497787,TIMED,2025-05-25T15:00:00Z,340,57,0,0
378,38,497788,TIMED,2025-05-25T15:00:00Z,73,397,0,0


In [25]:

write_deltalake(
    "data_lake/all_matches",
    matches_data,
    mode="overwrite"
)

In [26]:
all_matches_data_pq = DeltaTable("data_lake/all_matches").to_pandas()
all_matches_data_pq # Tabla delta con todos partidos de la premier league

,matchday,id,status,date,home,away,goals_home,goals_away
0,1,497410,TIMED,2024-08-16T19:00:00Z,66,63,0,0
1,1,497411,TIMED,2024-08-17T11:30:00Z,349,64,0,0
2,1,497412,TIMED,2024-08-17T14:00:00Z,57,76,0,0
3,1,497413,TIMED,2024-08-17T14:00:00Z,62,397,0,0
4,1,497414,TIMED,2024-08-17T14:00:00Z,67,340,0,0
...,...,...,...,...,...,...,...,...
375,38,497785,TIMED,2025-05-25T15:00:00Z,67,62,0,0
376,38,497786,TIMED,2025-05-25T15:00:00Z,351,61,0,0
377,38,497787,TIMED,2025-05-25T15:00:00Z,340,57,0,0
378,38,497788,TIMED,2025-05-25T15:00:00Z,73,397,0,0


### Extraccion incremental de los partidos de la premier league de la temporada 2024/2025
El archivo metadata.json guarda la fecha de la ultima carga de partidos finalizados, para poder hacer una extraccion incremental de los partidos de la premier league de la temporada 2024/2025

In [27]:
# update_last_update_in_json("metadata/metadata.json","2024-08-15T00:00:00Z") # Para actualizar un dia antes de la temporada
print(f"Ultima actualizacion { get_metadata_from_json("metadata/metadata.json") }")

Ultima actualizacion 2024-08-15T00:00:00Z


In [106]:
# EJECUTAR PARA ACTUALIZAR LOS DATOS

# Aca hago un get a los partidos que ya terminaron, y que cumplen que son mayores a la fecha que esta en el metadata.json
last_date = get_metadata_from_json("metadata/metadata.json")
last_date = datetime.strptime(last_date,'%Y-%m-%dT%H:%M:%SZ').strftime('%Y-%m-%d')
# Recolectamos solo año-mes-dia
params = {
    'dateFrom':last_date,
    # ATENCION
    #'dateTo':datetime.now().strftime('%Y-%m-%d') # Deberia ir este pero la premier todavia no arranco
    'dateTo' :"2024-10-12" # Fecha simulada de pruebas, se cargaran los partidos desde el dateFrom hasta el dateTo
    
}

played_matches = get_data(url_base,endpoint_matches,headers=headers,params=params)
played_matches['matches'][-1]

'2024-10-05'

In [93]:
# Procesar datos de los partidos jugados que no fueron cargados
played_matches_df = [
    dict(
            matchday=match['matchday'],
            id=match['id'],
            status=match['status'],
            date=match['utcDate'],
            home=match['homeTeam']['id'],
            away=match['awayTeam']['id'],
            goals_home=match['score']['fullTime']['home'],
            goals_away=match['score']['fullTime']['away']
    )
    for match in played_matches['matches']
]    

played_matches_df= pd.DataFrame(data=played_matches_df)

# played_matches_df = played_matches_df.query('status == "FINISHED"') ESTO DESACTIVADO PARA PRUEBAS, LOS PARTIDOS SE DEBERIAN CARGAR EN status == "FINISHED"

played_matches_df.fillna(0,inplace=True)
played_matches_df.head()
# Aca tendriamos los partidos ya jugados


,matchday,id,status,date,home,away,goals_home,goals_away
0,7,497470,SCHEDULED,2024-10-05T00:00:00Z,57,340,0,0
1,7,497471,SCHEDULED,2024-10-05T00:00:00Z,58,66,0,0
2,7,497472,SCHEDULED,2024-10-05T00:00:00Z,402,76,0,0
3,7,497473,SCHEDULED,2024-10-05T00:00:00Z,397,73,0,0
4,7,497474,SCHEDULED,2024-10-05T00:00:00Z,61,351,0,0


In [94]:
played_matches_df['date'].max() # Fecha maxima deL df, osea de los partidos jugados

'2024-10-05T00:00:00Z'

In [95]:
played_matches_df.tail()

,matchday,id,status,date,home,away,goals_home,goals_away
5,7,497475,SCHEDULED,2024-10-05T00:00:00Z,354,64,0,0
6,7,497476,SCHEDULED,2024-10-05T00:00:00Z,62,67,0,0
7,7,497477,SCHEDULED,2024-10-05T00:00:00Z,338,1044,0,0
8,7,497478,SCHEDULED,2024-10-05T00:00:00Z,65,63,0,0
9,7,497479,SCHEDULED,2024-10-05T00:00:00Z,563,349,0,0


 <p style="color:red;">NO VOLVER A EJECUTAR LA CELDA DE ABAJO SI YA SE CREO LA TABLA</p>

In [96]:

# Creamos df vacio para poder crear el delta lake
empty_df = pd.DataFrame(columns=['matchday','id','status','date','home','away','goals_home','goals_away'])
empty_df

# Deltalake no me permitia crear un deltalake con un dataframe vacio, se me ocurrio 
# la solucion de crear un dataframe con un solo registro y luego eliminarlo
# ChatGPT me ayudo con la logica de como hacerlo
schema = pa.schema([
    pa.field('matchday',pa.int64()),
    pa.field('id',pa.int64()),
    pa.field('status',pa.string()),
    pa.field('date',pa.string()),
    pa.field('home',pa.int64()),
    pa.field('away',pa.int64()),
    pa.field('goals_home',pa.int64()),
    pa.field('goals_away',pa.int64())
])

empty_df_pa = pa.Table.from_arrays([[]] * len(schema), schema=schema)

# Verificaciones para no tener errores
import shutil

if os.path.exists("data_lake/played_matches") == False:
     # Se crea la tabla delta particionada por matchday, esta estara vacia en el momento de su creacion
    write_deltalake(
        "data_lake/played_matches",
        data=empty_df_pa,
        mode="overwrite",
        partition_by=["matchday"]
    )
    
else: # Si la carpeta ya existe, se pregunta si se desea eliminar
    print("La carpeta ya existe")
    while True:
        dcs = input("Quieres eliminarla? Y/N")
        if dcs.lower() == "y" or dcs.lower() == "n":
            break
        else:
            print("Opcion invalida")
            
    if dcs.lower() == "y":
        shutil.rmtree("data_lake/played_matches")
        print("Carpeta eliminada correctamente")
        dcs = "n"
    else:
        print("No se elimino la carpeta")        
    

La carpeta ya existe
No se elimino la carpeta


In [97]:
# Insercion de datos al delta lake
try:
    if played_matches_df.empty:
        raise Exception("No hay datos para cargar")
    
    if os.path.exists("data_lake/played_matches") == False:
        raise Exception("No existe la tabla played_matches")
    
    dt = DeltaTable("data_lake/played_matches")

    new_data_pa = pa.Table.from_pandas(played_matches_df)
    (
        dt.merge(
            source=new_data_pa,
            source_alias="source",
            target_alias="target",
            predicate="source.id = target.id"
        )
        .when_not_matched_insert_all() 
        .execute()
    ) 
    # Se actualiza la metadata una vez insertados los datos
    update_last_update_in_json("metadata/metadata.json",played_matches_df['date'].max())
except Exception as e:
    print(f"Error: {e}")

In [98]:
# Aca podes ver los partidos jugados en la fecha que se indique
matchday = 7 # para ejemplo
try:
    ls = os.listdir(f"data_lake/played_matches/matchday={matchday}/")
    df = pd.read_parquet(f"data_lake/played_matches/matchday={matchday}/{ls[-1]}") # Muestra el ultimo parquet
except FileNotFoundError:
    print("No se encontro el directorio")

df.sort_values(by='date',ascending=False) 



,id,status,date,home,away,goals_home,goals_away
0,497478,SCHEDULED,2024-10-05T00:00:00Z,65,63,0,0
1,497477,SCHEDULED,2024-10-05T00:00:00Z,338,1044,0,0
2,497473,SCHEDULED,2024-10-05T00:00:00Z,397,73,0,0
3,497471,SCHEDULED,2024-10-05T00:00:00Z,58,66,0,0
4,497470,SCHEDULED,2024-10-05T00:00:00Z,57,340,0,0
5,497476,SCHEDULED,2024-10-05T00:00:00Z,62,67,0,0
6,497474,SCHEDULED,2024-10-05T00:00:00Z,61,351,0,0
7,497472,SCHEDULED,2024-10-05T00:00:00Z,402,76,0,0
8,497475,SCHEDULED,2024-10-05T00:00:00Z,354,64,0,0
9,497479,SCHEDULED,2024-10-05T00:00:00Z,563,349,0,0


In [99]:
df = pd.read_parquet(f"data_lake/played_matches")


print(f"Se cargaron {len(df)} partidos jugados en total")
df.tail() # Muestra los ultimos partidos jugados cargados

Se cargaron 70 partidos jugados en total


,id,status,date,home,away,goals_home,goals_away,matchday
65,497476,SCHEDULED,2024-10-05T00:00:00Z,62,67,0,0,7
66,497474,SCHEDULED,2024-10-05T00:00:00Z,61,351,0,0,7
67,497472,SCHEDULED,2024-10-05T00:00:00Z,402,76,0,0,7
68,497475,SCHEDULED,2024-10-05T00:00:00Z,354,64,0,0,7
69,497479,SCHEDULED,2024-10-05T00:00:00Z,563,349,0,0,7


### Data Testing

Con la libreria **Great Expectations**

In [100]:
df = gx.read_parquet("data_lake/played_matches") # Un dataframe
type(df) # -> great_expectations.dataset.pandas_dataset.PandasDataset
df.head() # Es un dataframe de great_expectations, que es como un pandas, pero con mas funcionalidades


,id,status,date,home,away,goals_home,goals_away,matchday
0,497417,TIMED,2024-08-18T13:00:00Z,402,354,0,0,1
1,497415,TIMED,2024-08-17T14:00:00Z,351,1044,0,0,1
2,497410,TIMED,2024-08-16T19:00:00Z,66,63,0,0,1
3,497419,TIMED,2024-08-19T19:00:00Z,338,73,0,0,1
4,497413,TIMED,2024-08-17T14:00:00Z,62,397,0,0,1


In [101]:
# Verificar que los id de los partidos sean unicos
if df.expect_column_values_to_be_unique(column='id')['success']:
    print("Los id de los partidos son unicos")

Los id de los partidos son unicos


In [103]:
# Verificar que todos los partidos cargados esten en estado FINISHED
x = df.expect_column_values_to_be_in_set(column='status',value_set=['FINISHED'])
if x['success']:
    print("Todos los partidos cargados estan en estado FINISHED")
else:
    print(f"Hay {x['result']['unexpected_count']} partidos que no estan en estado FINISHED")
    

Hay 70 partidos que no estan en estado FINISHED


In [104]:
# Verificar edad media de los jugadores

df = pd.read_parquet("data_lake/squads")

df['age'] = (datetime.now() - pd.to_datetime(df['dateOfBirth'])).dt.days // 365

print(f" Edad media de jugadores  {df['age'].mean()}")

 Edad media de jugadores  25.372013651877133
